## Install the packages

In [1]:
# %pip install nba_api
# %pip install pandas

## Import the libraries

In [22]:
import pandas as pd
import time

from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import playernextngames

## Basic search parameters

In [23]:
season = '2020-21'
season_type = 'Regular Season'

## Get active players IDs

In [24]:
active_players = players.get_active_players()
active_players[0:2]

[{'id': 1630173,
  'full_name': 'Precious Achiuwa',
  'first_name': 'Precious',
  'last_name': 'Achiuwa',
  'is_active': True},
 {'id': 203500,
  'full_name': 'Steven Adams',
  'first_name': 'Steven',
  'last_name': 'Adams',
  'is_active': True}]

In [25]:
players_ids = [str(player['id']) for player in active_players]
players_ids[0:2]

['1630173', '203500']

In [5]:
len(players_ids)

587

## Get teams IDs

In [26]:
nba_teams_full_info = teams.get_teams()
nba_teams_full_info[:2]

[{'id': 1610612737,
  'full_name': 'Atlanta Hawks',
  'abbreviation': 'ATL',
  'nickname': 'Hawks',
  'city': 'Atlanta',
  'state': 'Atlanta',
  'year_founded': 1949},
 {'id': 1610612738,
  'full_name': 'Boston Celtics',
  'abbreviation': 'BOS',
  'nickname': 'Celtics',
  'city': 'Boston',
  'state': 'Massachusetts',
  'year_founded': 1946}]

In [27]:
nba_teams_short_info = [{'id': team['id'], 'full_name': team['full_name'], 'abbreviation': team['abbreviation']} for team in nba_teams_full_info]
nba_teams_short_info[0:2]

[{'id': 1610612737, 'full_name': 'Atlanta Hawks', 'abbreviation': 'ATL'},
 {'id': 1610612738, 'full_name': 'Boston Celtics', 'abbreviation': 'BOS'}]

In [28]:
nba_teams_ids = [team['id'] for team in nba_teams_full_info]
nba_teams_ids[0:2]

[1610612737, 1610612738]

## Get stats by game (player total)

In [10]:
columns = ['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE']

df = pd.DataFrame(columns=columns)

In [11]:
for idx, id in enumerate(players_ids):
    try:
        data = playergamelog.PlayerGameLog(
            player_id=id,
            season=season,
            season_type_all_star=season_type,
            league_id_nullable='00',
            timeout=10)
    
        df = pd.concat([df, data.get_data_frames()[0]])
        
        # Preventing timeout exceptions
        time.sleep(.600)
    except:
        print(f'There was some problem while gathering data.\nPlayer id: {id}\nIteration #{idx}\n')

df.head(3)

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22020,1630173,0022001069,"MAY 16, 2021",MIA @ DET,W,42,10,16,0.625,...,7,10,0,2,1,2,3,23,6,1
1,22020,1630173,0022001062,"MAY 15, 2021",MIA @ MIL,L,2,1,2,0.5,...,0,0,1,0,0,0,0,2,3,1
2,22020,1630173,0022001050,"MAY 13, 2021",MIA vs. PHI,W,2,0,0,0.0,...,0,0,0,0,0,0,0,0,-5,1


In [12]:
df.shape

(21349, 27)

In [17]:
print(f"The final dataset contais stats about {df['Player_ID'].nunique()} players")

The final dataset contais stats about 453 players


This difference in relation to the length of players_ids is acceptable, due various reasons, like the players who dont play any game during the season, or players who got hurt in the pre-season.

## Save the DataFrame

In [7]:
# Pickle
# df.to_pickle('df_all_players_raw.pkl')

# Unpickle
df = pd.read_pickle('df_all_players_raw.pkl')

## Adjust the columns names

In [8]:
df.columns

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'],
      dtype='object')

In [9]:
new_column_names = ['Season ID', 'Player ID', 'Game ID', 'Game Date', 'Matchup', 'Won or Lost',
       'Minutes played', 'Field Goals Made', 'Field Goals Attempted', 'Field Goals %', 
       'Field Goals 3 Points Made', 'Field Goal 3 Points Attempt', 'Field Goal 3 Points %', 
       'Free Throws Made', 'Free Throws Attempt', 'Free Throw %', 'Offensive Rebounds', 'Defensive Rebounds', 'Rebounds', 
       'Assists', 'Steals', 'Blocks', 'Turnovers', 'Personal Fouls',
       'Points', 'Plus Minus', 'Video Available']
    
df.columns = new_column_names
df.columns

Index(['Season ID', 'Player ID', 'Game ID', 'Game Date', 'Matchup',
       'Won or Lost', 'Minutes played', 'Field Goals Made',
       'Field Goals Attempted', 'Field Goals %', 'Field Goals 3 Points Made',
       'Field Goal 3 Points Attempt', 'Field Goal 3 Points %',
       'Free Throws Made', 'Free Throws Attempt', 'Free Throw %',
       'Offensive Rebounds', 'Defensive Rebounds', 'Rebounds', 'Assists',
       'Steals', 'Blocks', 'Turnovers', 'Personal Fouls', 'Points',
       'Plus Minus', 'Video Available'],
      dtype='object')

## Drop the irrelevant features

In [10]:
columns_to_drop = ['Field Goals %', 'Field Goal 3 Points %', 
'Free Throw %', 'Rebounds', 'Personal Fouls', 'Plus Minus', 'Video Available']

df.drop(columns=columns_to_drop, inplace=True)
df.columns

Index(['Season ID', 'Player ID', 'Game ID', 'Game Date', 'Matchup',
       'Won or Lost', 'Minutes played', 'Field Goals Made',
       'Field Goals Attempted', 'Field Goals 3 Points Made',
       'Field Goal 3 Points Attempt', 'Free Throws Made',
       'Free Throws Attempt', 'Offensive Rebounds', 'Defensive Rebounds',
       'Assists', 'Steals', 'Blocks', 'Turnovers', 'Points'],
      dtype='object')

In [11]:
df.head(2)

,Season ID,Player ID,Game ID,Game Date,Matchup,Won or Lost,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,Field Goal 3 Points Attempt,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Points
0,22020,1630173,0022001069,"MAY 16, 2021",MIA @ DET,W,42,10,16,0,0,3,7,3,7,0,2,1,2,23
1,22020,1630173,0022001062,"MAY 15, 2021",MIA @ MIL,L,2,1,2,0,1,0,0,0,0,1,0,0,0,2


## Checking the missing values

In [12]:
df.isna().sum().sum()

0

## Check for duplicated rows

In [13]:
df.duplicated().sum()

0

## Reset the index

In [14]:
df.reset_index(drop=True, inplace=True)

## Adjust the data types

### Adjust Game Date

In [15]:
df['Game Date'] =  pd.to_datetime(df['Game Date'])
df.head(2)

,Season ID,Player ID,Game ID,Game Date,Matchup,Won or Lost,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,Field Goal 3 Points Attempt,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Points
0,22020,1630173,0022001069,2021-05-16,MIA @ DET,W,42,10,16,0,0,3,7,3,7,0,2,1,2,23
1,22020,1630173,0022001062,2021-05-15,MIA @ MIL,L,2,1,2,0,1,0,0,0,0,1,0,0,0,2


### Adjust Won or Lost

In [16]:
df['Won or Lost'].replace({'W': 1, 'L': 0}, inplace=True)
df.rename(columns={'Won or Lost': 'Won'}, inplace=True)
df.head(2)

,Season ID,Player ID,Game ID,Game Date,Matchup,Won,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,Field Goal 3 Points Attempt,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Points
0,22020,1630173,0022001069,2021-05-16,MIA @ DET,1,42,10,16,0,0,3,7,3,7,0,2,1,2,23
1,22020,1630173,0022001062,2021-05-15,MIA @ MIL,0,2,1,2,0,1,0,0,0,0,1,0,0,0,2


### Adjust the numeric features types

In [17]:
df.dtypes

Season ID                              object
Player ID                              object
Game ID                                object
Game Date                      datetime64[ns]
Matchup                                object
Won                                     int64
Minutes played                         object
Field Goals Made                       object
Field Goals Attempted                  object
Field Goals 3 Points Made              object
Field Goal 3 Points Attempt            object
Free Throws Made                       object
Free Throws Attempt                    object
Offensive Rebounds                     object
Defensive Rebounds                     object
Assists                                object
Steals                                 object
Blocks                                 object
Turnovers                              object
Points                                 object
dtype: object

In [18]:
columns_to_int = ['Minutes played', 'Field Goals Made', 'Field Goals Attempted',
       'Field Goals 3 Points Made', 'Field Goal 3 Points Attempt',
       'Free Throws Made', 'Free Throws Attempt', 'Offensive Rebounds',
       'Defensive Rebounds', 'Assists', 'Steals', 'Blocks', 'Turnovers',
       'Points']

columns_to_str = ['Season ID', 'Player ID', 'Game ID', 'Matchup']

In [19]:
df[columns_to_int] = df[columns_to_int].apply(pd.to_numeric)
df[columns_to_str] = df[columns_to_str].apply(lambda x: x.astype(str))
df.dtypes

Season ID                              object
Player ID                              object
Game ID                                object
Game Date                      datetime64[ns]
Matchup                                object
Won                                     int64
Minutes played                          int64
Field Goals Made                        int64
Field Goals Attempted                   int64
Field Goals 3 Points Made               int64
Field Goal 3 Points Attempt             int64
Free Throws Made                        int64
Free Throws Attempt                     int64
Offensive Rebounds                      int64
Defensive Rebounds                      int64
Assists                                 int64
Steals                                  int64
Blocks                                  int64
Turnovers                               int64
Points                                  int64
dtype: object

In [20]:
df.head(2)

,Season ID,Player ID,Game ID,Game Date,Matchup,Won,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,Field Goal 3 Points Attempt,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Points
0,22020,1630173,0022001069,2021-05-16,MIA @ DET,1,42,10,16,0,0,3,7,3,7,0,2,1,2,23
1,22020,1630173,0022001062,2021-05-15,MIA @ MIL,0,2,1,2,0,1,0,0,0,0,1,0,0,0,2


## Include aditional features

### Include player name

In [29]:
players_ids[0:3]

['1630173', '203500', '1628389']

In [30]:
active_players[0:2]

[{'id': 1630173,
  'full_name': 'Precious Achiuwa',
  'first_name': 'Precious',
  'last_name': 'Achiuwa',
  'is_active': True},
 {'id': 203500,
  'full_name': 'Steven Adams',
  'first_name': 'Steven',
  'last_name': 'Adams',
  'is_active': True}]

In [31]:
players_ids.index('203500')

1

In [25]:
active_players[players_ids.index('203500')]['full_name']

'Steven Adams'

In [32]:
df['Player Name'] = df['Player ID'].apply(lambda x: str(active_players[players_ids.index(str(x))]['full_name']))
df.head(2)

,Season ID,Player ID,Game ID,Game Date,Matchup,Won,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,...,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Points,Player Name
0,22020,1630173,0022001069,2021-05-16,MIA @ DET,1,42,10,16,0,...,3,7,3,7,0,2,1,2,23,Precious Achiuwa
1,22020,1630173,0022001062,2021-05-15,MIA @ MIL,0,2,1,2,0,...,0,0,0,0,1,0,0,0,2,Precious Achiuwa


In [33]:
df.tail(2)

,Season ID,Player ID,Game ID,Game Date,Matchup,Won,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,...,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Points,Player Name
21347,22020,1627826,0022000009,2020-12-25,LAC @ DEN,1,19,2,2,0,...,8,11,0,0,0,0,1,2,12,Ivica Zubac
21348,22020,1627826,0022000002,2020-12-22,LAC @ LAL,1,27,4,5,0,...,3,4,3,3,1,1,1,1,11,Ivica Zubac


### Include the players team name

#### Games

In [34]:
game_finder = leaguegamefinder.LeagueGameFinder(season_nullable=season, season_type_nullable=season_type)
games = game_finder.get_data_frames()[0]
games.head(2)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22020,1610612758,SAC,Sacramento Kings,0022001077,2021-05-16,SAC vs. UTA,L,241,99,...,0.600,5,34,39,24,10,5,12,22,-22.0
1,22020,1610612743,DEN,Denver Nuggets,0022001076,2021-05-16,DEN @ POR,L,240,116,...,0.933,10,26,36,20,8,2,6,20,-16.0


In [35]:
games = games[['SEASON_ID', 'TEAM_ABBREVIATION', 'GAME_ID', 'WL']]
games.head(2)

,SEASON_ID,TEAM_ABBREVIATION,GAME_ID,WL
0,22020,SAC,0022001077,L
1,22020,DEN,0022001076,L


In [36]:
def get_team_name(players_df, player_id, games_df, game_id):
    if players_df[(players_df['Player ID'] == player_id) & (players_df['Game ID'] == game_id)]['Won'].values[0]:
        return games_df[(games_df['GAME_ID'] == game_id) & (games_df['WL'] == 'W')]['TEAM_ABBREVIATION'].values[0]
    else:
        return games_df[(games_df['GAME_ID'] == game_id) & (games_df['WL'] == 'L')]['TEAM_ABBREVIATION'].values[0]

In [37]:
# Player ID = 1627826: Ivica Zubac
# Expected return: LAC
get_team_name(df, '1627826', games, '0022000002')

'LAC'

In [38]:
df['Player Team'] = ''

for idx, row in df.iterrows():
    df['Player Team'].iloc[idx] = get_team_name(df, row['Player ID'], games, row['Game ID'])

C:\Users\danie\AppData\Local\Temp\ipykernel_17780\3590590045.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Player Team'].iloc[idx] = get_team_name(df, row['Player ID'], games, row['Game ID'])


In [39]:
df[(df['Player ID'] == '1627826') & (df['Won'] == 0)].head(1)

,Season ID,Player ID,Game ID,Game Date,Matchup,Won,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,...,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Points,Player Name,Player Team
21277,22020,1627826,0022001074,2021-05-16,LAC @ OKC,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Ivica Zubac,LAC


In [40]:
df[df['Player Name'] == 'LeBron James'].head(1)

,Season ID,Player ID,Game ID,Game Date,Matchup,Won,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,...,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Points,Player Name,Player Team
9873,22020,2544,0022001072,2021-05-16,LAL @ NOP,1,27,11,22,2,...,2,1,0,6,3,0,2,25,LeBron James,LAL


### Include Home x Away

In [41]:
df[df['Game ID'] == '0022001072']

,Season ID,Player ID,Game ID,Game Date,Matchup,Won,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,...,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Points,Player Name,Player Team
209,22020,1629638,0022001072,2021-05-16,NOP vs. LAL,0,37,5,15,2,...,2,1,3,5,1,0,4,14,Nickeil Alexander-Walker,NOP
2231,22020,202339,0022001072,2021-05-16,NOP vs. LAL,0,22,5,9,2,...,2,0,4,2,0,2,0,13,Eric Bledsoe,NOP
3608,22020,203484,0022001072,2021-05-16,LAL @ NOP,1,27,4,8,2,...,0,0,1,6,1,0,0,10,Kentavious Caldwell-Pope,LAL
4008,22020,1627936,0022001072,2021-05-16,LAL @ NOP,1,19,4,8,1,...,0,1,2,5,0,1,1,9,Alex Caruso,LAL
4918,22020,203076,0022001072,2021-05-16,LAL @ NOP,1,30,5,12,0,...,6,0,5,1,1,0,1,14,Anthony Davis,LAL
5560,22020,203083,0022001072,2021-05-16,LAL @ NOP,1,21,6,11,0,...,3,5,8,0,2,2,2,13,Andre Drummond,LAL
6342,22020,1629117,0022001072,2021-05-16,NOP vs. LAL,0,20,1,4,0,...,2,0,4,0,0,1,0,4,Wenyen Gabriel,NOP
8207,22020,1629637,0022001072,2021-05-16,NOP vs. LAL,0,20,4,7,0,...,2,2,5,3,1,0,0,10,Jaxson Hayes,NOP
8389,22020,1626195,0022001072,2021-05-16,NOP vs. LAL,0,30,8,11,0,...,5,4,9,1,0,2,2,19,Willy Hernangomez,NOP
9024,22020,1629659,0022001072,2021-05-16,LAL @ NOP,1,14,2,6,1,...,2,0,1,1,2,0,1,6,Talen Horton-Tucker,LAL


#### The '@' in the 'Matchup' feature indicates that the game was in the second location

In [42]:
df['Home'] = df['Matchup'].apply(lambda x: 0 if x[4] == '@' else 1)

In [43]:
df[df['Game ID'] == '0022001072'][['Game ID', 'Matchup', 'Player Name', 'Player Team', 'Home']]

,Game ID,Matchup,Player Name,Player Team,Home
209,0022001072,NOP vs. LAL,Nickeil Alexander-Walker,NOP,1
2231,0022001072,NOP vs. LAL,Eric Bledsoe,NOP,1
3608,0022001072,LAL @ NOP,Kentavious Caldwell-Pope,LAL,0
4008,0022001072,LAL @ NOP,Alex Caruso,LAL,0
4918,0022001072,LAL @ NOP,Anthony Davis,LAL,0
5560,0022001072,LAL @ NOP,Andre Drummond,LAL,0
6342,0022001072,NOP vs. LAL,Wenyen Gabriel,NOP,1
8207,0022001072,NOP vs. LAL,Jaxson Hayes,NOP,1
8389,0022001072,NOP vs. LAL,Willy Hernangomez,NOP,1
9024,0022001072,LAL @ NOP,Talen Horton-Tucker,LAL,0


### Include opponent

In [44]:
df['Opponent'] = df['Matchup'].apply(lambda x: x[-3:])
df.head()

,Season ID,Player ID,Game ID,Game Date,Matchup,Won,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,...,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Points,Player Name,Player Team,Home,Opponent
0,22020,1630173,0022001069,2021-05-16,MIA @ DET,1,42,10,16,0,...,7,0,2,1,2,23,Precious Achiuwa,MIA,0,DET
1,22020,1630173,0022001062,2021-05-15,MIA @ MIL,0,2,1,2,0,...,0,1,0,0,0,2,Precious Achiuwa,MIA,0,MIL
2,22020,1630173,0022001050,2021-05-13,MIA vs. PHI,1,2,0,0,0,...,0,0,0,0,0,0,Precious Achiuwa,MIA,1,PHI
3,22020,1630173,0022000986,2021-05-04,MIA vs. DAL,0,6,1,2,0,...,1,1,0,1,0,2,Precious Achiuwa,MIA,1,DAL
4,22020,1630173,0022000969,2021-05-02,MIA @ CHA,1,1,0,1,0,...,0,0,0,0,0,0,Precious Achiuwa,MIA,0,CHA


### Rearrange columns order

In [45]:
df.columns

Index(['Season ID', 'Player ID', 'Game ID', 'Game Date', 'Matchup', 'Won',
       'Minutes played', 'Field Goals Made', 'Field Goals Attempted',
       'Field Goals 3 Points Made', 'Field Goal 3 Points Attempt',
       'Free Throws Made', 'Free Throws Attempt', 'Offensive Rebounds',
       'Defensive Rebounds', 'Assists', 'Steals', 'Blocks', 'Turnovers',
       'Points', 'Player Name', 'Player Team', 'Home', 'Opponent'],
      dtype='object')

In [46]:
new_columns_order = ['Season ID', 'Player ID', 'Player Name',
                    'Game ID', 'Game Date', 'Matchup', 'Player Team', 'Opponent', 'Home', 'Won',
                    'Minutes played', 'Field Goals Made', 'Field Goals Attempted',
                    'Field Goals 3 Points Made', 'Field Goal 3 Points Attempt',
                    'Free Throws Made', 'Free Throws Attempt', 'Offensive Rebounds',
                    'Defensive Rebounds', 'Assists', 'Steals', 'Blocks', 'Turnovers',
                    'Points']

In [3]:
df = df[new_columns_order]
df.head(2)

,Season ID,Player ID,Player Name,Game ID,Game Date,Matchup,Player Team,Opponent,Home,Won,...,Field Goal 3 Points Attempt,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Points
0,22020,1630173,Precious Achiuwa,0022001069,2021-05-16,MIA @ DET,MIA,DET,0,1,...,0,3,7,3,7,0,2,1,2,23
1,22020,1630173,Precious Achiuwa,0022001062,2021-05-15,MIA @ MIL,MIA,MIL,0,0,...,1,0,0,0,0,1,0,0,0,2


## Save the DataFrame

In [2]:
import pandas as pd

# Pickle
# df.to_pickle('final_df.pkl')

# Unpickle
df = pd.read_pickle('final_df.pkl')

## Get the next games

In [49]:
playernextngames.PlayerNextNGames(
    number_of_games=3,
    player_id=players_ids[0], 
    season_all='2021-22', 
    season_type_all_star=season_type).get_data_frames()[0]

,GAME_ID,GAME_DATE,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_NAME,VISITOR_TEAM_NAME,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_NICKNAME,GAME_TIME,HOME_WL,VISITOR_WL
0,0022100790,"FEB 04, 2022",1610612761,1610612737,Toronto,Atlanta,TOR,ATL,Raptors,Hawks,07:30 PM,27-23,25-26
1,0022100810,"FEB 07, 2022",1610612766,1610612761,Charlotte,Toronto,CHA,TOR,Hornets,Raptors,07:00 PM,28-24,27-23
2,0022100827,"FEB 09, 2022",1610612760,1610612761,Oklahoma City,Toronto,OKC,TOR,Thunder,Raptors,08:00 PM,16-34,27-23
